<a href="https://colab.research.google.com/github/NeSma237/collaborative-filtering-recommender/blob/main/notebooks/user_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir collaborative-filtering-recommender
!cd collaborative-filtering-recommender
!mkdir data notebooks src results
!touch README.md requirements.txt
!touch README.md requirements.txt

In [5]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

import pandas as pd

ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')
users = pd.read_csv('ml-100k/u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

--2025-04-25 17:44:53--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.53MB/s    in 0.8s    

2025-04-25 17:44:54 (5.53 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader
!pip install numpy==1.23.5 --force-reinstall
!pip install scikit-surprise


In [26]:
user_item_matrix = pd.read_csv("data/merged_data.csv", index_col='user_id')
print("User-Item Matrix Shape:", user_item_matrix.shape)
print("Sample (First 5 Users, First 5 Movies):")
print(user_item_matrix.iloc[:5, :5])

User-Item Matrix Shape: (100000, 30)
Sample (First 5 Users, First 5 Movies):
         item_id  rating  timestamp                       title release_date
user_id                                                                     
196          242       3  881250949                Kolya (1996)  24-Jan-1997
186          302       3  891717742    L.A. Confidential (1997)  01-Jan-1997
22           377       1  878887116         Heavyweights (1994)  01-Jan-1994
244           51       2  880606923  Legends of the Fall (1994)  01-Jan-1994
166          346       1  886397596         Jackie Brown (1997)  01-Jan-1997


In [2]:
# Load the original ratings data (not the user-item matrix)
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp']) # Changed the file path to 'ml-100k/u.data'

# Define the rating scale (1-5)
reader = Reader(rating_scale=(1, 5))

# Load data into Surprise format
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

# Split into train-test (80-20)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
# Correct way to check sizes:

print(f"Train set size: {trainset.n_ratings} ratings")
print(f"Test set size: {len(testset)} ratings")

Train set size: 80000 ratings
Test set size: 20000 ratings


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader
#!pip install numpy==1.23.5 --force-reinstall
#!pip install scikit-surprise
# Calculate cosine similarity between users
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')

# Merge ratings and movies data on 'movie_id'
data = pd.merge(ratings, movies, left_on='movie_id', right_on='movie_id') # Specifying left_on and right_on

# Create user-item matrix
user_item_matrix = data.pivot_table(
    index='user_id',
    columns='movie_title',  # Changed to 'movie_title' assuming it's the movie identifier
    values='rating',
    fill_value=0
)

# Now you can calculate the cosine similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

print("User Similarity Matrix Shape:", user_similarity.shape)
print("Sample (First 5 Users):")
print(user_similarity.iloc[:5, :5])


User Similarity Matrix Shape: (943, 943)
Sample (First 5 Users):
user_id         1         2         3         4         5
user_id                                                  
1        1.000000  0.168937  0.048388  0.064561  0.379670
2        0.168937  1.000000  0.113393  0.179694  0.073623
3        0.048388  0.113393  1.000000  0.349781  0.021592
4        0.064561  0.179694  0.349781  1.000000  0.031804
5        0.379670  0.073623  0.021592  0.031804  1.000000


In [6]:
def predict_rating(user_id, movie_title, k=5):

    # Get similarity scores for the target user
    sim_scores = user_similarity.loc[user_id]

    # Get ratings for the target movie
    movie_ratings = user_item_matrix[movie_title]

    # Remove users who haven't rated the movie
    valid_users = movie_ratings[movie_ratings > 0].index
    sim_scores = sim_scores[valid_users]

    # Get top-K most similar users
    top_k_users = sim_scores.sort_values(ascending=False)[1:k+1]  # Exclude self

    # Calculate weighted average
    weighted_sum = np.dot(
        top_k_users.values,
        user_item_matrix.loc[top_k_users.index, movie_title]
    )
    prediction = weighted_sum / top_k_users.sum()

    return prediction

In [7]:
user_id = 1
movie_title = "Toy Story (1995)"
predicted_rating = predict_rating(user_id, movie_title, k=5)

print(f"Predicted rating for User {user_id} on '{movie_title}': {predicted_rating:.2f}")

Predicted rating for User 1 on 'Toy Story (1995)': 4.00


In [8]:
def recommend_movies(user_id, n=5):
    """
    Recommends top-N movies a user hasn't rated yet.
    """
    # Movies the user has already rated
    rated_movies = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index

    # Predict ratings for unrated movies
    predictions = []
    for movie in user_item_matrix.columns:
        if movie not in rated_movies:
            pred = predict_rating(user_id, movie)
            predictions.append((movie, pred))

    # Sort by predicted rating
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

    return recommendations

# Example: Get top 5 recommendations for User 1
user_id = 1
recommendations = recommend_movies(user_id, n=5)

print(f"Top 5 Recommendations for User {user_id}:")
for i, (movie, rating) in enumerate(recommendations, 1):
    print(f"{i}. {movie} (Predicted Rating: {rating:.2f})")

<ipython-input-6-aa40b963a197>:21: RuntimeWarning: invalid value encountered in double_scalars
  prediction = weighted_sum / top_k_users.sum()


Top 5 Recommendations for User 1:
1. Kundun (1997) (Predicted Rating: 4.59)
2. 8 Seconds (1994) (Predicted Rating: 4.50)
3. 8 1/2 (1963) (Predicted Rating: 4.39)
4. Boogie Nights (1997) (Predicted Rating: 4.21)
5. African Queen, The (1951) (Predicted Rating: 4.01)


In [9]:
def batch_predict(testset, user_similarity, user_item_matrix, movies, k=5):

    # Create mapping dictionaries for faster lookups
    movie_id_to_title = dict(zip(movies['item_id'], movies['title']))
    user_mean_ratings = user_item_matrix.mean(axis=1)
    global_mean = user_item_matrix.mean().mean()

    predictions = []

    # Pre-compute all possible movie titles in test set
    test_movies = {movie for _, movie, _ in testset}
    available_movies = set(movie_id_to_title.keys())

    for user, movie, _ in testset:
        # Skip if movie not in our training data
        if movie not in available_movies:
            predictions.append(global_mean)
            continue

        movie_title = movie_id_to_title[movie]

        try:
            # Get similar users (excluding self)
            sim_scores = user_similarity.loc[user]
            user_ratings = user_item_matrix[movie_title]

            # Find users who rated this movie
            rated_users = user_ratings[user_ratings > 0].index
            rated_users = rated_users[rated_users != user]  # Exclude self

            if len(rated_users) == 0:
                predictions.append(user_mean_ratings.loc[user])
                continue

            # Get top K similar users who rated this movie
            user_sims = sim_scores[rated_users]
            top_k_users = user_sims.nlargest(k).index

            # Calculate weighted average
            weighted_sum = (user_item_matrix.loc[top_k_users, movie_title] *
                          user_similarity.loc[user, top_k_users]).sum()
            norm = user_similarity.loc[user, top_k_users].sum()

            prediction = weighted_sum / norm if norm != 0 else user_mean_ratings.loc[user]
            predictions.append(prediction)

        except:
            predictions.append(global_mean)

    return predictions


In [14]:
def precision_recall_at_k(user_id, ratings_df, movies_df, k=5, threshold=3.5):

    # Get ground truth (movies the user rated highly)
    user_ratings = ratings_df[ratings_df['user_id'] == user_id]
    # Changed 'item_id' to 'movie_id' to match the column name in ratings_df
    highly_rated = set(user_ratings[user_ratings['rating'] >= threshold]['movie_id'])

    # Get recommendations (assuming recommend_movies exists)
    recommended_movies = recommend_movies(user_id, n=k)

    # Convert recommendations to item IDs
    recommended = []
    for movie_title, _ in recommended_movies:
        # Assuming 'movie_id' is the correct column in movies_df
        movie_id = movies_df[movies_df['movie_title'] == movie_title]['movie_id'].values
        if len(movie_id) > 0:
            recommended.append(movie_id[0])

    # Calculate metrics
    relevant_and_recommended = len(highly_rated.intersection(recommended))
    precision = relevant_and_recommended / k
    recall = relevant_and_recommended / len(highly_rated) if highly_rated else 0

    return precision, recall

# Example usage:

ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')


# Then evaluate for a user
precision, recall = precision_recall_at_k(
    user_id=1,
    ratings_df=ratings,
    movies_df=movies,
    k=5
)
print(f"Precision@5: {precision:.2f}, Recall@5: {recall:.2f}")

<ipython-input-6-aa40b963a197>:21: RuntimeWarning: invalid value encountered in double_scalars
  prediction = weighted_sum / top_k_users.sum()


Precision@5: 0.00, Recall@5: 0.00
